# Creating Redshift cluster using the AWS python SDK

In [20]:
import pandas as pd
import boto3
import json
import configparser
from botocore.exceptions import ClientError

In [14]:
config = configparser.ConfigParser()
config.read('dwh.cfg')

KEY                   = config.get('AWS', 'KEY')
SECRET                = config.get('AWS', 'SECRET')

DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")

DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")

pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER", "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER, DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

,Param,Value
0,DWH_CLUSTER_TYPE,multi-node
1,DWH_NUM_NODES,4
2,DWH_NODE_TYPE,dc2.large
3,DWH_CLUSTER_IDENTIFIER,redshift-cluster-1
4,DWH_DB,dev
5,DWH_DB_USER,awsuser
6,DWH_DB_PASSWORD,9507024922
7,DWH_PORT,5439
8,DWH_IAM_ROLE_NAME,myRedshiftRole


### Create clients for IAM, EC2, S3 and Redshift

#### Note:This resources are being created in the us-east-1 region

In [16]:
ec2 = boto3.resource('ec2', 
                    region_name = "us-west-2",
                   aws_access_key_id = KEY,
                   aws_secret_access_key=SECRET)
s3 = boto3.resource('s3',
                    region_name = "us-west-2",
                   aws_access_key_id = KEY,
                   aws_secret_access_key=SECRET)
iam = boto3.client('iam',
                   region_name = "us-west-2",
                   aws_access_key_id = KEY,
                   aws_secret_access_key=SECRET)
redshift = boto3.client('redshift',
                        region_name = "us-west-2",
                   aws_access_key_id = KEY,
                   aws_secret_access_key=SECRET)

In [19]:
s3bucket = s3.Bucket("udacity-dend")

s3_logs = iter(s3bucket.objects.filter(Prefix = "log-data"))
for _ in range(10):
    print(next(s3_logs))

s3_songs = iter(s3bucket.objects.filter(Prefix = "song_data"))
for _ in range(10):
    print(next(s3_songs))

s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-01-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-02-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-03-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-04-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-05-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-06-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-07-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-08-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-09-events.json')
s3.ObjectSummary(bucket_name='udacity-dend', key='song_data/')
s3.ObjectSummary(bucket_name='udacity-dend', ke

In [ ]:
#1.1 Create the Redshift Role

try:
    print("1.1 Creating a IAM Role : myRedshiftRole")
    dwhRole = iam.create_role(
        Path = '/',
        RoleName = DWH_IAM_ROLE_NAME,
        Description = "Allows Redshift clusters to call AWS services on your behalf.",
        AssumeRolePolicyDocument=json.dumps(
                {'Statement': ["Allows Redshift cluster to call AWS services on your behalf"]
                    
                })
            )